In [2]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from queue import Queue
filepath_out='../output/bench1.router'
netlist_file_path = '../benchmark/bench1.nl'
gridfile_path  = '../benchmark/bench1.grid'

rows,columns,bend_penalty,via_penalty,layer1_gird,layer2_grid = parse_gridfile(gridfile_path)
nets,net_num = parse_netlist(netlist_file_path)
print(net_num)

bend_penalty: 5 
 via_penalty: 1
size of layer1: (50, 50)
size of layer2: (50, 50)
20


In [3]:
def unit_cost_router(rows, columns, layer1_grid, nets):
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = -1
        layer1_grid[pin2['x']][pin2['y']] = -1
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
        
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        # 将需要布线的两个pin重新标回可布线状态
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path = expand_source_to_target(rows, columns, layer1_grid, pin1, pin2)
        mark_path_on_grid(layer1_grid,path)
        routing_table[net_id] = path
    return routing_table



def expand_source_to_target(rows, columns, layer1_grid, source, target):
    queue = Queue()
    visited = set()
    parents = {}

    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    queue.put(source_tuple)
    visited.add(source_tuple)

    while not queue.empty():
        current_cell = queue.get()

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                if layer1_grid[neighbor['x']][neighbor['y']] != -1:
                    queue.put(neighbor_tuple)
                    visited.add(neighbor_tuple)
                    parents[neighbor_tuple] = current_cell

    return []

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})

    return neighbors


def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1

In [4]:
routing_table = unit_cost_router(rows, columns, layer1_gird, nets)
print(routing_table)
generate_output_file(filepath_out, net_num, routing_table)

{1: [(3, 36, 1), (3, 35, 1), (3, 34, 1), (3, 33, 1), (3, 32, 1), (3, 31, 1), (3, 30, 1), (3, 29, 1)], 2: [(4, 36, 1), (4, 35, 1), (4, 34, 1), (4, 33, 1), (4, 32, 1), (4, 31, 1), (4, 30, 1), (4, 29, 1)], 3: [(5, 36, 1), (6, 36, 1), (6, 35, 1), (6, 34, 1), (6, 33, 1), (6, 32, 1), (6, 31, 1), (6, 30, 1), (6, 29, 1)], 4: [(7, 36, 1), (8, 36, 1), (9, 36, 1), (9, 35, 1), (9, 34, 1), (9, 33, 1), (9, 32, 1), (9, 31, 1), (9, 30, 1), (9, 29, 1)], 5: [(10, 36, 1), (11, 36, 1), (12, 36, 1), (13, 36, 1), (13, 35, 1), (13, 34, 1), (13, 33, 1), (13, 32, 1), (13, 31, 1), (13, 30, 1), (13, 29, 1)], 6: [(49, 36, 1), (48, 36, 1), (47, 36, 1), (46, 36, 1), (45, 36, 1), (44, 36, 1), (43, 36, 1), (42, 36, 1), (41, 36, 1), (40, 36, 1), (39, 36, 1), (38, 36, 1), (37, 36, 1), (36, 36, 1)], 7: [(49, 35, 1), (48, 35, 1), (47, 35, 1), (46, 35, 1), (45, 35, 1), (44, 35, 1), (43, 35, 1), (42, 35, 1), (41, 35, 1), (40, 35, 1), (39, 35, 1), (38, 35, 1), (37, 35, 1), (36, 35, 1)], 8: [(49, 34, 1), (48, 34, 1), (47, 34